In [256]:
import collections
import itertools
import functools
import math
# import multiprocessing
import operator
import string
# import random

import numba
import numpy as np
# import pandas as pd

from nltk import tokenize
from nltk.util import ngrams

# from scipy.spatial.distance import jaccard
# from scipy.special import expit as sigmoid

# from sklearn.metrics import jaccard_similarity_score
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import euclidean_distances, cosine_distances

In [257]:
import json
import pathlib

cwd = pathlib.Path.cwd()
data = cwd / 'data'
jsons = data / 'jsons'
json_2018 = jsons / '2018'

json_2018 = list(json_2018.iterdir())[0]

In [258]:
with open(json_2018) as fp:
    articles_2018 = json.load(fp)['2018']

article = articles_2018[4]
text = article['story']
summ_true = article['summary']
# print(text)

# Evolutionary Algorithms
- [Discrete Differential Evolution for Text Summarization](https://www.researchgate.net/publication/281662415_Discrete_Differential_Evolution_for_Text_Summarization)
- [Evolutionary Algorithm for Extractive Text Summarization](https://www.researchgate.net/profile/Ramiz_Aliguliyev/publication/220518077_Evolutionary_Algorithm_for_Extractive_Text_Summarization/links/09e4151356fc2caab6000000.pdf)
- [An Improved Evolutionary Algorithm for Extractive Text Summarization](https://link.springer.com/chapter/10.1007/978-3-642-36543-0_9)

# Similarity Measures

### Jaccard Coefficient Similarity Measure

$
\large
sim_{ jaccard } ( S_i, S_j ) = 
\dfrac
    { | S_i \cap S_j | }
    { | S_i \cup S_j | }
$

In [259]:
@numba.njit
def jaccard_similarity(a, b):
    #: assume union is non-empty since each sentence >= 1 word
    return np.sum(a & b) / np.sum(a | b)

# Objective Functions

Let $ C $ be a partition of $ D $ with $ k $ clusters.  
$ C = \{ C_1, C_2, ..., C_k \} $  

where:
- $ C_p \cap C_q = \emptyset, 
        \forall p \ne q \in \{ 1, 2, ..., k \} 
  $
- $ \bigcup\limits
    _{ p = 1 }
    ^k C_p = D 
  $
- $ C_p \ne \emptyset, 
        \forall p \in \{ 1, 2, ..., k \}
  $

### Sigmoid Function
$
\large
sigm ( x ) = 
\dfrac
    { 1 }
    { 1 + \text{exp} ( -x ) }
$

In [260]:
@numba.njit
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

### Intra-Cluster Similiarity (Cohesion)

$
\large
F_1 = 
\sum\limits
    _{ \small p = 1 }
    ^{ \small k } 
\sum\limits
    _{ \small S_i, S_j \in C_p } 
\dfrac 
    { sim ( S_i, S_j ) } 
    { | C_p | } 
\rightarrow \text{max}
$

*(__Note:__ Evol Alg for Ext Txt Summ doesn't show division of $|C_p|$ but I believe this to be a typo since the paragraph detailing it says "the average sum". Additionally the Disc Diff Evol for Txt Summ shows it as such.)*

In [304]:
# @numba.njit
# def cohesion(chromosome, similarity, document):
#     total = 0
#     for p in np.unique(chromosome):
#         sents = document[np.where(chromosome == p)]
#         k = len(sents)
#         #: combinations choose 2
#         for i in range(k-1):
#             for j in range(i+1, k):
#                 total += similarity(sents[i], sents[j]) / len(sents)  
#     return total


@numba.njit(parallel=True)
def cohesion(chromosome, similarity, document):
    total = 0
    clusters = len(np.unique(chromosome))
    for p in numba.prange(clusters):
        sents = document[np.where(chromosome == p)]
        k = len(sents)
        #: combinations choose 2
        for i in numba.prange(k-1):
            for j in numba.prange(i+1, k):
                total += similarity(sents[i], sents[j]) / len(sents)  
    return total

### Inter-Cluster Dissimilarity (Separation)

$
\large
F_2 = 
\sum\limits
    _{ \small p = 1 }
    ^{ \small k - 1 }
\sum\limits
    _{ \small q = p + 1 }
    ^{ \small k } 
\sum\limits
    _{ \small S_i \in C_p } 
\sum\limits
    _{ \small S_j \in C_q } 
\dfrac 
    { sim ( S_i, S_j ) } 
    { | C_p | \cdot | C_q | }
\rightarrow \text{min}
$

In [305]:
# @numba.njit
# def separation(chromosome, similarity, document):
#     total = 0
#     k = len(np.unique(chromosome))
#     #: combinations choose 2
#     for p in range(k-1):
#         for q in range(p+1, k):
#             sents_p = document[np.where(chromosome == p)]
#             sents_q = document[np.where(chromosome == q)]
#             #: product
#             m, n = len(sents_p), len(sents_q)
#             for i in range(m):
#                 for j in range(n):
#                     total += similarity(sents_p[i], sents_q[j]) / m / n
#     return total


@numba.njit(parallel=True)
def separation(chromosome, similarity, document):
    total = 0
    k = len(np.unique(chromosome))
    #: combinations choose 2
    for p in numba.prange(k-1):
        for q in numba.prange(p+1, k):
            sents_p = document[np.where(chromosome == p)]
            sents_q = document[np.where(chromosome == q)]
            #: product
            m, n = len(sents_p), len(sents_q)
            for i in numba.prange(m):
                for j in numba.prange(n):
                    total += similarity(sents_p[i], sents_q[j]) / m / n
    return total

### Inter/Intra-Cluster Balance

$
\large
F = \big( 1 + sigm ( F_1 ) \big)
    ^{ F_2 } \rightarrow \text{max}
$

In [306]:
@numba.njit
def cohesion_separation(chromosome, similarity, document):
    coh = cohesion(chromosome, similarity, document)
    sep = separation(chromosome, similarity, document)
    return (1 + sigmoid(coh)) ** sep

## Fitness

$
\large
fitness_1 \big( X_a ( t ) \big) = F_1 \big( X_a ( t ) \big)
$

$
\large
fitness_2 \big( X_a ( t ) \big) = \dfrac{ 1 }{ F_2 ( X_a ( t ) ) }
$

$
\large
fitness \big( X_a ( t ) \big) = F \big( X_a ( t ) \big)
$

# Modified Discrete Differential Evolution Algorithm

Initialize the population with $ N $ chromosomes each composed of $ n $ random integers from \[1, k\].  

$
X_r ( t ) = [ x_{ r, 1 } ( t ), x_{ r, 2 } ( t ), ..., x_{ r, n } ( t ) ]
$  

where:
- $ x_{ r, s } ( t ) \in \{ 1, 2, ..., k \} $
- $ r = 1, 2, ..., N $
- $ s = 1, 2, ..., n $
- $ N $ is the population size
- $ n $ is the number of sentences _(in the document)_
- $ k $ is the number of clusters _(number of sentences for summary)_
- $ t $ is the iteration step

In [275]:
def init_chromosome(choices, length):
    chrom = np.full(length, -1)
    #: ensure that each choice is accounted for at least once
    idxs = np.random.choice(np.arange(length), len(choices), replace=False)
    chrom[idxs] = np.random.permutation(choices)
    idxs = np.where(chrom == -1)[0]
    chrom[idxs] = np.random.choice(choices, len(idxs))
    return chrom


def init_population(population_size, cluster_amount, chromosome_length):
    clusts = np.arange(cluster_amount)
    chroms = [init_chromosome(clusts, chromosome_length) for _ in range(population_size)]
    pop = np.vstack(chroms)
    return pop

$
\large
y_{ r, s } ( t + 1 ) = 
\begin{cases}
    x_{ r1, s } ( t ) + \lambda \big( x_{ r2, s } ( t ) - x_{ r3, s } ( t ) \big) 
        & \text{if } rand_s < \text{CR} \\
    x_{ r, s } ( t ) & \text{otherwise}
\end{cases}
$

where  
- For each $ X_r $, randomly sample $ X_{ r1 } ( t ), X_{ r2 } ( t ), X_{ r3 } ( t ) $ 
  from the same generation _(each distinct)_
- $ rand_s $ is uniformally distributed random numbers from $ [ 0, 1 ] $ 
  chosen once for each $ s \in \{ 1, 2, ..., n \} $

hyper-parameters 
- $ \lambda $ is a scale factor from $ [ 0, 1 ] $
- $ \text{CR} $ is the crossover rate from $ [ 0, 1 ] $

In [276]:
def get_offspring(population, randoms, lambda_, crossover_rate):
    #: For computation time, relax requirement that X_r, X_r1, X_r2, X_r3 are distinct. 
    #: With large population size, this is unlikely to occur, and if it does, it doesn't
    #: seem that detrimental. Also is this mitigated with appropriate lam choice?
    n = len(population)
    idxs = np.random.choice(np.arange(n), size=(n, 3))
    chrom_1, chrom_2, chrom_3 = map(np.squeeze, np.split(population[idxs], 3, axis=1))
    k = len(np.unique(population))
    offspr = (chrom_1 + lambda_ * (chrom_2 - chrom_3)) % k
    mask = randoms < crossover_rate
    offspr[mask] = population[mask]
    return offspr

In [277]:
#TODO: early stopping --> little fitness improvement over x generations, good enough fitness score
def run_iterations(pop_size, summ_len, num_sents, func, lam, cr, iterations, *, mutate_after=True,
                   seed=None, verbose=False, save_rate=np.nan, save_dir=None):
    
    if save_dir is not None:
        save_dir = pathlib.Path(save_dir)
        if not save_dir.is_dir():
            msg = f'save_dir={save_dir} not a valid directory path'.format(save_dir=save_dir)
            raise NotADirectoryError(msg)
    
    if seed is not None:
        np.random.seed(seed)
    
    pop = init_population(pop_size, summ_len, num_sents)
    shape = pop.shape
    for i in range(iterations):
        if i % save_rate == 0:
            file = save_dir / 'generation_{i:0>pad}'.format(i=i, pad=len(str(iterations)))
            np.save(file, pop)
            
        if verbose:
            print(i)  #TODO: logfile --> iteration number, best fitness score, avg fitness score, hyper-params
        
        rand = np.random.random_sample(shape)
        
        t0 = time.time()
        offspr = get_offspring(pop, rand, lam, cr)
        t1 = time.time()
        PROFILER['offspring'] += t1 - t0
        
        #: option since papers unclear if mutate at offspring or survivors stage
        if not mutate_after:
            mutate(offspr, rand)
            
        t0 = time.time()
        next_generation(pop, offspr, func)
        t1 = time.time()
        PROFILER['generation'] += t1 - t0
        
        if mutate_after:
            t0 = time.time()
            mutate(pop, rand)
            t1 = time.time()
            PROFILER['mutate'] += t1 - t0
    
    return pop

In [278]:
def best_chromosome(population):
    #TODO: make sure it picks one with all k-clusters
    fits = np.argmax([fitness(chrom) for chrom in population])
    chrom = population[fits]
    return chrom
    

def central_sentences(chromosome, document, metric=cosine_distances):
    central_sents = []
    for cluster in np.unique(chromosome):
        idxs = np.where(chromosome == cluster)[0]
        sents = document[idxs]
        centroid = sents.mean(axis=0)[np.newaxis,:]
        dists = metric(sents, centroid)
        cent_sent = idxs[np.argmin(dists)]
        central_sents.append(cent_sent)
    return central_sents

$
\large
X_r(t+1) = 
\begin{cases}
    Y_r ( t + 1 ) & \text{if } f \big( Y_r ( t + 1 ) \big) > f \big( X_r ( t ) \big) \\
    X_r ( t ) & \text{otherwise}
\end{cases}
$

where 
- $ f ( \cdot ) $ is the objective function to be maximized

In [290]:
def next_generation(population, offspring, func):
    fit_off = np.array([func(chrom) for chrom in offspring])
    fit_pop = np.array([func(chrom) for chrom in population])
    mask = fit_off > fit_pop
    population[mask] = offspring[mask]
    return

# @numba.njit(parallel=True)
# def _get_mask(pop, offspr, func):
#     fit_pop = np.empty_like(pop)
#     fit_off = np.empty_like(offspr)
#     for i in numba.prange(len(pop)):
#         fit_pop[i] = func(pop[i])
#         fit_off[i] = func(offspr[i])
#     mask = fit_off > fit_pop
#     return mask

# def next_generation(pop, offspr, func):
#     mask = _get_mask(pop, offspr, func)
#     pop[mask] = offspr[mask]
#     return

## Mutation

At each iteration $ t + 1 $ for each $ X_r ( t ) $ creates
$ m_r ( t + 1 ) = [ m_{ r, 1 } ( t ), m_{ r, 2 } ( t ), ..., m_{ r, n } ( t ) ] $.  
For each gene, 1 indicates no mutation and 0 means mutate.

$
\large
m_{ r, s } ( t + 1 ) = 
\begin{cases}
    1 & \text{if } rand_s < sigm \big( y_{ r, s } ( t + 1 ) \big) \\
    0 & \text{otherwise}
\end{cases}
$

### Inversion Operator

### Figure 1 - Psuedo-code
<img src="./data/pngs/fig1_-_inversion_operator_psuedo_code.png" alt="Fig 1. Inverse operator psuedo-code" width="33%" align="left"/>

### Figure 2 - Example
<img src="./data/pngs/fig2_-_inversion_operator_diagram.png" alt="Fig 2. Inverse operator example" width="33%" align="left"/>

In [287]:
def mutate(population, randoms):
    mask = randoms < sigmoid(population)
    #: inversion operator
    idxs = np.nonzero(mask)
    arr = np.array(idxs)
    sorter = np.lexsort((-arr[1], arr[0]))
    rev = arr.T[sorter].T
    population[idxs] = population[(rev[0], rev[1])]
    return

# ROUGE-N
_(Recall Oriented Understudy for Gisting Evaluation)_

$
\large
\text{ROUGE-N} = 
\dfrac
    { \sum \limits
        _{ \small S \in Summ_{ ref }} 
        \sum \limits
        _{ \small \text{N-gram} \in S } 
            Count_{ \small match } ( \text{N-gram} ) }
    { \sum \limits
        _{ \small S \in Summ_{ ref } } 
        \sum \limits
            _{ \small \text{N-gram} \in S } 
            Count( \text{N-gram} ) }
$

In [288]:
def rouge_n(n, y_pred, y_true):
    n_gram_pred = set(ngrams(y_pred, n))
    n_gram_true = set(ngrams(y_true, n))
    return len(n_gram_pred & n_gram_true) / len(n_gram_true)

# Run Algorithm

In [307]:
import time

PROFILER = collections.Counter()


def fitness(chromosome):
    return cohesion_separation(chromosome, jaccard_similarity, doc)

cv = CountVectorizer(stop_words='english')
sents = tokenize.sent_tokenize(text.lower())
vec = cv.fit_transform(sents)
doc = vec.toarray().astype(bool).astype(int)


t0 = time.time()
pop = run_iterations(pop_size=100, summ_len=5, num_sents=len(doc), 
                     func=fitness, lam=0.9, cr=0.5, iterations=500, verbose=False, seed=0)
t1 = time.time()
print(t1-t0)


chrom_best = best_chromosome(pop)
idxs = central_sentences(chrom_best, doc)
summ_evol = '\n'.join(np.array(tokenize.sent_tokenize(text))[idxs])

PROFILER

35.624605894088745


Counter({'offspring': 0.15188932418823242,
         'generation': 35.24577760696411,
         'mutate': 0.18845772743225098})

In [303]:
print(summ_evol)

Antos and Corlette both say they don't expect the availability of short-term policies to have a major impact on the ACA exchanges.
Many of those people have been priced out of the health insurance market since the ACA took effect, says Joseph Antos, an economist at the American Enterprise Institute.
Short-term plans don't have to meet the Affordable Care Act's consumer protection and coverage requirements, so many will not cover services such as mental health care or prescription drugs.
People who don't get insurance through their jobs will now be able to buy short-term policies that may be cheaper than Affordable Care Act coverage.
The plans are aimed at people who earn more than four times the federal poverty threshold and therefore don't qualify for federal subsidies to help pay premiums for policies sold on the ACA exchanges.


In [295]:
print(summ_evol)

And insurance companies can deny customers coverage on these plans if they have a pre-existing medical condition and charge people more if they are likely to need medical care.
Many of those people have been priced out of the health insurance market since the ACA took effect, says Joseph Antos, an economist at the American Enterprise Institute.
Short-term plans don't have to meet the Affordable Care Act's consumer protection and coverage requirements, so many will not cover services such as mental health care or prescription drugs.
And if a short-term policy ends before the open enrollment period for Affordable Care Act insurance, consumers will not have the right to buy ACA insurance right away, says Randy Pate, the deputy administrator of the Centers for Medicare and Medicaid Services at HHS.
According to a report by the National Association of Insurance Commissioners, the policies paid out an average 55 percent of their premiums in actual health care last year.


In [ ]:
def scores(n, y_pred):
    for i in range(1, n+1):
        score = rouge_n(i, y_pred, summ_true)
        print(i, score)

In [188]:
print('evolutionary')
scores(5, summ_evol)

evolutionary
1 0.9230769230769231
2 0.8529411764705882
3 0.5827338129496403
4 0.46938775510204084
5 0.3959731543624161


In [186]:
import gensim

summ_gensim = gensim.summarization.summarize(text)
print('gensim')
scores(5, summ_gensim)

gensim
1 0.9615384615384616
2 0.9411764705882353
3 0.7410071942446043
4 0.6258503401360545
5 0.5100671140939598


In [185]:
from sumy.parsers.plaintext import PlaintextParser
from sumy.summarizers.lex_rank import LexRankSummarizer
from sumy.summarizers.text_rank import TextRankSummarizer
from sumy.nlp.tokenizers import Tokenizer


def format_sumy(sumy_summary):
    return ''.join(char for sent in sumy_summary for char in str(sent) if char in string.printable)

parser = PlaintextParser(text, Tokenizer('english'))

text_rank = TextRankSummarizer()
summ_text_rank = format_sumy(text_rank(parser.document, 5))
print('sumy - text rank')
scores(5, summ_text_rank)

print()

lex_rank = LexRankSummarizer()
summ_lex_rank = format_sumy(lex_rank(parser.document, 2))
print('sumy - lex rank')
scores(5, summ_lex_rank)

sumy - text rank
1 0.9615384615384616
2 0.8529411764705882
3 0.6330935251798561
4 0.5034013605442177
5 0.4228187919463087

sumy - lex rank
1 0.9230769230769231
2 0.7254901960784313
3 0.48201438848920863
4 0.35374149659863946
5 0.26174496644295303
